## csv 데이터 기반 전처리

In [121]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

class AISPreprocessor:
    # data_dir: 경로 데이터 파일(csv)이 존재하는 폴더 이름 
    # input_seq_len: 참조할 이전 과거 정보(default: 10분)
    # output_seq_len: 예측할 미래 정보(default: 1분)
    def __init__(self, data_dir, input_seq_len=10, output_seq_len=1):
        self.data_dir = data_dir
        self.input_seq_len = input_seq_len
        self.output_seq_len = output_seq_len

        # 수동 설정된 범위로 MinMaxScaler 초기화
        # 정규화 범위 설정
        lat_range = (33.0, 38.0)
        lon_range = (124.0, 132.0)
        sog_range = (0.0, 100.0)
        cog_range = (0.0, 360.0)
        heading_range = (0.0, 360.0)
        
        # MinMaxScaler 수동 설정
        self.scaler = MinMaxScaler()
        self.scaler.min_ = np.array([
            -lat_range[0] / (lat_range[1] - lat_range[0]),
            -lon_range[0] / (lon_range[1] - lon_range[0]),
            -sog_range[0] / (sog_range[1] - sog_range[0]),
            -cog_range[0] / (cog_range[1] - cog_range[0]),
            -heading_range[0] / (heading_range[1] - heading_range[0])
        ])
        self.scaler.scale_ = np.array([
            1 / (lat_range[1] - lat_range[0]),
            1 / (lon_range[1] - lon_range[0]),
            1 / (sog_range[1] - sog_range[0]),
            1 / (cog_range[1] - cog_range[0]),
            1 / (heading_range[1] - heading_range[0])
        ])
        self.scaler.feature_names_in_ = np.array(['위도', '경도', 'SOG', 'COG', 'Heading'])


    def load_and_preprocess(self):
        input_seqs = []
        output_seqs = []
        count = 1
        for file in os.listdir(self.data_dir):
            if file.endswith('.csv'):
                print(f"---------- {count}번째 파일 진행 중 ----------")
                count += 1
                df = pd.read_csv(os.path.join(self.data_dir, file), encoding='cp949')
                df = self._preprocess_single_file(df)
                in_seqs, out_seqs = self._extract_sequences(df)
                input_seqs.extend(in_seqs)
                output_seqs.extend(out_seqs)

        return np.array(input_seqs), np.array(output_seqs)

    def _preprocess_single_file(self, df):
        df = df[['일시', '위도', '경도', 'SOG', 'COG', 'Heading']].copy()
        df['일시'] = pd.to_datetime(df['일시'])
        # 일시 기준 데이터 sorting
        df = df.sort_values('일시')
        # NA 데이터 drop
        df = df.dropna()
        
        # 각 데이터를, 일시 기준 1분 별 보간법을 적용해 transform 적용 ------------------------------------
        # 예: 01:00:04, 01:00:06, 01:00:16 데이터 -> 평균 -> 01:00:00(1시 0분)으로 한 샘플 생성
        # 에: 01:01:05, 01:01:12, 01:01:35, 01:01:44 데이터 -> 평균 -> 01:01:00(1시 1분)으로 한 샘플 생성
        # 평균을 적용하는 데이터는 위도, 경도, sog, cog 데이터 
        df = df.set_index('일시').resample('1min').mean().interpolate()
        df = df.reset_index()
        # -----------------------------------------------------------------------------------------------
        
        # 목적지 좌표: 마지막 위치
        dest_lat = df['위도'].iloc[-1]
        dest_lon = df['경도'].iloc[-1]
        
        # feature engineering을 위한 feature 저장 
        df['dest_lat'] = dest_lat
        df['dest_lon'] = dest_lon
        return df

    def _extract_sequences(self, df):
        input_seqs = []
        output_seqs = []
    
        total_len = self.input_seq_len + self.output_seq_len
        for i in range(len(df) - total_len):
            input_window = df.iloc[i:i+self.input_seq_len]
            output_window = df.iloc[i+self.input_seq_len:i+total_len]
    
            # 입력: 위도, 경도, SOG, COG, Heading (정규화)
            input_scaled = self.scaler.transform(input_window[['위도', '경도', 'SOG', 'COG', 'Heading']])
            input_seq = input_scaled
            # 출력: 위도, 경도, SOG, COG, Heading (정규화)
            output_seq = self.scaler.transform(output_window[['위도', '경도', 'SOG', 'COG', 'Heading']])
    
            input_seqs.append(input_seq)
            output_seqs.append(output_seq)
    
        return input_seqs, output_seqs

In [122]:
import warnings
warnings.filterwarnings(action='ignore')
# 데이터 디렉토리 경로 설정 (예: routes 폴더 안에 여러 개의 csv가 있는 경우)
data_dir = './routes'
# 전처리 객체 생성
preprocessor = AISPreprocessor(data_dir)

# 데이터 로드 및 전처리 실행
input_seqs, output_seqs = preprocessor.load_and_preprocess()

# 출력 확인
print("Input sequences shape:", input_seqs.shape)   # (num_samples, 10, 5)
print("Output sequences shape:", output_seqs.shape) # (num_samples, 1, 5)

# 예시 데이터 출력 (첫 샘플)
print("\nSample Input Sequence (첫 번째 샘플):")
print(input_seqs[5])

print("\nSample Output Sequence (첫 번째 샘플):")
print(output_seqs[5])

---------- 1번째 파일 진행 중 ----------
---------- 2번째 파일 진행 중 ----------
---------- 3번째 파일 진행 중 ----------
---------- 4번째 파일 진행 중 ----------
---------- 5번째 파일 진행 중 ----------
---------- 6번째 파일 진행 중 ----------
---------- 7번째 파일 진행 중 ----------
---------- 8번째 파일 진행 중 ----------
---------- 9번째 파일 진행 중 ----------
---------- 10번째 파일 진행 중 ----------
---------- 11번째 파일 진행 중 ----------
---------- 12번째 파일 진행 중 ----------
---------- 13번째 파일 진행 중 ----------
---------- 14번째 파일 진행 중 ----------
---------- 15번째 파일 진행 중 ----------
---------- 16번째 파일 진행 중 ----------
---------- 17번째 파일 진행 중 ----------
---------- 18번째 파일 진행 중 ----------
---------- 19번째 파일 진행 중 ----------
---------- 20번째 파일 진행 중 ----------
---------- 21번째 파일 진행 중 ----------
---------- 22번째 파일 진행 중 ----------
---------- 23번째 파일 진행 중 ----------
---------- 24번째 파일 진행 중 ----------
---------- 25번째 파일 진행 중 ----------
---------- 26번째 파일 진행 중 ----------
---------- 27번째 파일 진행 중 ----------
---------- 28번째 파일 진행 중 ----------
---------- 29번째 파일 진행 중 -----

## 경로 예측모델 

- 입력 시퀀스 (위도, 경도, SOG, COG, (목적지-현재위치 위도), (목적지-현재위치 경도))
  
      ↓
  
- Linear 프로젝션 (input_size → d_model)

  
      ↓

  
- Positional Encoding 추가

  
      ↓

  
- Transformer Encoder (Multi-head Self Attention, FeedForward)

  
      ↓

  
- Decoder (MLP)

  
      ↓

  
- 출력 (예: 다음 시점의 위도, 경도, SOG, COG)

In [124]:
!pip install tensorboard

In [125]:
from torch.utils.tensorboard import SummaryWriter

In [126]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# 위도/경도 오차 부분에 중점을 두기 위한 사용자 설정 LOSS FUNCTION
class CustomTrajectoryLoss(nn.Module):
    def __init__(self, weight_location = 0.8):
        super().__init__()
        self.weight_location = weight_location

    def forward(self, y_pred, y_true):
        y_pred = y_pred.squeeze(1)
        y_true = y_true.squeeze(1)
        # y_pred, y_true: (batch_size, 5) → [lat, lon, SOG, COG, Heading]
        pred_lat, pred_lon = y_pred[:, 0], y_pred[:, 1]
        true_lat, true_lon = y_true[:, 0], y_true[:, 1]
        pred_sog, pred_cog = y_pred[:, 2], y_pred[:, 3]
        true_sog, true_cog = y_true[:, 2], y_true[:, 3]
        pred_heading = y_pred[:, 4]
        true_heading = y_pred[:, 4]
        
        # 1. 위도/경도 loss
        location_loss = 0.5*torch.square(pred_lat-true_lat) + 0.5*torch.square(pred_lon-true_lon)
        location_loss = location_loss.mean()
        # 2. sog, cog loss
        else_loss = 0.5*torch.square(pred_sog-true_sog) + 0.5*torch.square(pred_cog-true_cog)
        else_loss = else_loss.mean()
        # 3. heading 손실 (360도 각도 차이 고려)
        heading_diff = torch.abs(pred_heading - true_heading)
        heading_diff = torch.where(heading_diff > 180, 360 - heading_diff, heading_diff)
        heading_loss = heading_diff.square().mean()
        
        # 4. 합산 Loss
        total_loss =  total_loss = (
            self.weight_location * location_loss +
            0.5 * (1 - self.weight_location) * else_loss +
            0.5 * (1 - self.weight_location) * heading_loss
    )
        return total_loss


In [127]:
import torch
import torch.nn as nn
## 위치 정보 전달을 위한 정적 포지셔널 인코딩
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # even index
        pe[:, 1::2] = torch.cos(position * div_term)  # odd index
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        x = x + self.pe[:, :x.size(1)]
        return x


### 시계열 학습을 위한 트랜스포머 회귀 모델
## d_model: 주목할 input의 특징들
## nhead: 멀티 헤드 어텐션 헤드 수
## dim_feedforward: FFN 차원 수
# dim_feedforward = d_model * 4
# d_model % n_head = 0
class TransformerPredictor(nn.Module):
    def __init__(self, input_size=5, output_size=5, d_model=512, nhead=8, num_layers=4, dim_feedforward=1024, dropout=0.1):
        super(TransformerPredictor, self).__init__()

        self.input_proj = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model) # 포지셔널 인코딩을 통해 순서 정보를 추가
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, # input의 특징들
            nhead=nhead, # 멀티 헤드 어텐션 헤드 수
            dim_feedforward=dim_feedforward, # FFN 차원 수, 기본 2048
            dropout=dropout,
            batch_first=True,
            activation="gelu", # default="relu"
        )
        # 인코더
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # MLP 기반 디코더
        self.decoder = nn.Sequential(
            nn.Linear(d_model, 512),
            nn.GELU(),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        x = self.input_proj(x)  # (batch_size, seq_len, d_model)
        x = self.pos_encoder(x)  # (batch_size, seq_len, d_model)
        x = self.transformer_encoder(x)  # (batch_size, seq_len, d_model)
        # 자가회귀 예측: 마지막 타임스텝의 출력만 사용
        x_last = x[:, -1, :]  # (batch_size, d_model)
        out = self.decoder(x_last)  # (batch_size, output_size)

        # 차원을 맞추기 위해 seq_len=1 축을 다시 추가
        out = out.unsqueeze(1)  # (batch_size, 1, output_size)
        return out

In [128]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

def train_transformer_model(model, train_data, val_data=None, num_epochs=50, batch_size=128, learning_rate=1e-4, device='cpu'):
    """
    model: TransformerPredictor 모델
    train_data: (x_train_tensor, y_train_tensor)
    val_data: (x_val_tensor, y_val_tensor)
    """
    model.to(device)
    
    ## train_data를 train / validation data로 분할 --------------------------------
    x_train, y_train = train_data
    x_train_f, x_val, y_train_f, y_val = train_test_split(
        x_train, y_train, test_size=0.2, random_state=42
    )

    
    train_dataset = TensorDataset(x_train_f, y_train_f)
    val_data = (x_val, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # ----------------------------------------------------------------------------
    
    # Loss & Optimizer
    #criterion = nn.MSELoss()
    criterion = CustomTrajectoryLoss(weight_location=0.95)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)

    
    # 로그 저장 디렉토리 설정
    writer = SummaryWriter(log_dir='./runs/transformer_experiment')

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    
            optimizer.zero_grad()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1) # gradient exploding 방지 
            outputs = model(batch_x)  # (batch, seq_len, output_size)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"[Epoch {epoch+1}/{num_epochs}] Train Loss: {avg_loss:.6f}")

        # 로그파일에 기록
        writer.add_scalar('Loss/Train', avg_loss, epoch)


        # Validation 진행
        if val_data is not None:
            model.eval()
            with torch.no_grad():
                x_val, y_val = val_data
                x_val, y_val = x_val.to(device), y_val.to(device)
                val_outputs = model(x_val)
                val_loss = criterion(val_outputs, y_val)
                print(f"           ↳ Val Loss: {val_loss.item():.6f}")
                # 로그파일에 기록
                writer.add_scalar('Loss/Val', val_loss.item(), epoch)

    writer.close()

In [129]:
# 모델 생성
model = TransformerPredictor(input_size=5, output_size=5)

# numpy → torch tensor로 변환
input_tensor = torch.tensor(input_seqs, dtype=torch.float32)
output_tensor = torch.tensor(output_seqs, dtype=torch.float32)
# 학습
train_transformer_model(model, (input_tensor, output_tensor), num_epochs=50, device='cuda' if torch.cuda.is_available() else 'cpu')

[Epoch 1/50] Train Loss: 0.004445
           ↳ Val Loss: 0.000184
[Epoch 2/50] Train Loss: 0.000220
           ↳ Val Loss: 0.000162
[Epoch 3/50] Train Loss: 0.000134
           ↳ Val Loss: 0.000206
[Epoch 4/50] Train Loss: 0.000091
           ↳ Val Loss: 0.000064
[Epoch 5/50] Train Loss: 0.000073
           ↳ Val Loss: 0.000118
[Epoch 6/50] Train Loss: 0.000060
           ↳ Val Loss: 0.000050
[Epoch 7/50] Train Loss: 0.000058
           ↳ Val Loss: 0.000056
[Epoch 8/50] Train Loss: 0.000044
           ↳ Val Loss: 0.000108
[Epoch 9/50] Train Loss: 0.000045
           ↳ Val Loss: 0.000060
[Epoch 10/50] Train Loss: 0.000035
           ↳ Val Loss: 0.000067
[Epoch 11/50] Train Loss: 0.000035
           ↳ Val Loss: 0.000048
[Epoch 12/50] Train Loss: 0.000032
           ↳ Val Loss: 0.000137
[Epoch 13/50] Train Loss: 0.000031
           ↳ Val Loss: 0.000052
[Epoch 14/50] Train Loss: 0.000028
           ↳ Val Loss: 0.000025
[Epoch 15/50] Train Loss: 0.000035
           ↳ Val Loss: 0.000035
[Epo

## 자가회귀 예측 코드

In [131]:
# 모델 저장
torch.save(model, "route_predictor2.pth")

In [132]:
import torch
import numpy as np

## 자가회귀를 위한 함수
## 선박의 예상 경로가 목적지 인근 부근이 될 때까지 모델의 예측 반복 / 임계값: max_steps
def predict_autoregressive(model, initial_seq, dest_lat, dest_lon, scaler, max_steps, distance_threshold=0.1):
    """
    model: 학습된 Transformer 모델
    initial_seq: 초기 입력 시퀀스 (torch.Tensor), shape: (1, 10, 5), 정규화된 값이 들어와야 됨.
    dest_lat, dest_lon: 목적지 좌표
    scaler: 학습에 사용한 MinMaxScaler
    device: 'cpu' or 'cuda'
    max_steps: 최대 예측 스텝 수
    distance_threshold: 도착지와 거리 차가 이 값 이하이면 도착으로 간주
    """
    model.eval()
    input_seq = initial_seq.clone()  # input_seq의 복사 생성
    all_preds = []

    for step in range(max_steps):
        input_tensor = input_seq
        # 예측
        with torch.no_grad():
            pred = model(input_tensor).squeeze(0).cpu().numpy()  # (seq_len, output_size)

        all_preds.append(pred)

        ## 예상 경로 도착지 도달 여부 확인 매커니즘 ----------------------------------------------------------
        # 예측한 경로값 역정규화
        pred_denorm = scaler.inverse_transform(pred.reshape(1, -1))[0][:5]
        pred_lat, pred_lon = pred_denorm[:2]

        # 디버깅 코드
        if(step % 10 == 0):
            print(f"[Step {step+1}] Predicted: ({pred_lat:.5f}, {pred_lon:.5f}) | "
                  f"Target: ({dest_lat:.5f}, {dest_lon:.5f}) | "
                  f"ΔLat: {abs(pred_lat - dest_lat):.5f}, ΔLon: {abs(pred_lon - dest_lon):.5f}")
    
    
        if abs(pred_lat - dest_lat) < distance_threshold and abs(pred_lon - dest_lon) < distance_threshold:
            print(f"🚢 목적지 도달 - Step: {step + 1}, {int(step/60)} 시간 {step%60} 분 소요")
            break

        ## ------------------------------------------------------------------------------------------------
            
        ## 입력 시퀀스 업데이트: 다음 입력을 만듦 -----------------------------------------------
        
        # 정규화된 lat/lon/sog/cog/heading (5개만 사용)
        pred_norm = pred.reshape(1, -1)[0]  # (5,)
        next_input = pred_norm
        # ---------------------------------------
        # 기존 시퀀스에서 가장 앞 데이터 제거, 새 데이터 추가
        input_seq_np = input_seq.squeeze(0).numpy()
        input_seq_np = np.vstack([input_seq_np[1:], next_input])
        input_seq = torch.tensor([input_seq_np], dtype=torch.float32)  # (1, 10, 5)
        # ----------------------------------------------------------------------
    return np.array(all_preds)

In [133]:
from math import atan2, sqrt, degrees

## 초기 10분 시퀀스
# 예시: 특정 CSV 파일에서 인천항 출발 경로 하나 로드
pre = AISPreprocessor(data_dir='routes', input_seq_len=10, output_seq_len=1)

df = pd.read_csv('routes/route_1_202002.csv', encoding='cp949', parse_dates=['일시'])
df = pre._preprocess_single_file(df)
initial_seq = df.iloc[:10]

# MinMaxScaler 수동 설정 ----------------------------------------------------------
# 정규화 범위 설정
lat_range = (33.0, 38.0)
lon_range = (124.0, 132.0)
sog_range = (0.0, 100.0)
cog_range = (0.0, 360.0)
heading_range = (0.0, 360.0)

input_scaler = MinMaxScaler()
input_scaler.min_ = np.array([
    -lat_range[0] / (lat_range[1] - lat_range[0]),
    -lon_range[0] / (lon_range[1] - lon_range[0]),
    -sog_range[0] / (sog_range[1] - sog_range[0]),
    -cog_range[0] / (cog_range[1] - cog_range[0]),
    -heading_range[0] / (heading_range[1] - heading_range[0])
])
input_scaler.scale_ = np.array([
    1 / (lat_range[1] - lat_range[0]),
    1 / (lon_range[1] - lon_range[0]),
    1 / (sog_range[1] - sog_range[0]),
    1 / (cog_range[1] - cog_range[0]),
    1 / (heading_range[1] - heading_range[0])
])
input_scaler.feature_names_in_ = np.array(['위도', '경도', 'SOG', 'COG', 'Heading'])
# --------------------------------------------------------------------------------

# 목적지 좌표 설정 - 제주항
dest_lat = 33.55  
dest_lon = 126.55  

# 입력 시퀀스 생성
test_input_seq = []
for _, row in initial_seq.iterrows():
    # 1. 정규화된 위도, 경도, SOG, COG
    scaled = input_scaler.transform([[row['위도'], row['경도'], row['SOG'], row['COG'], row['Heading']]])[0]

    # 4. 최종 입력 벡터 구성 (5차원)
    input_row = list(scaled)
    test_input_seq.append(input_row)

test_input_seq = torch.tensor([test_input_seq], dtype=torch.float32)  # (1, 10, 5)

# 목적지 좌표 설정
destination_lat = dest_lat
destination_lon = dest_lon
scaler = input_scaler

# 예측 실행
preds = predict_autoregressive(
    model=model,                      # 학습된 Transformer 모델
    initial_seq=test_input_seq,       # 초기 입력 시퀀스 
    dest_lat=destination_lat,         # 목적지 위도
    dest_lon=destination_lon,         # 목적지 경도
    scaler=scaler,                    # 학습에 사용된 MinMaxScaler                 
    max_steps=5000                    # 예측할 시간 길이 
)

[Step 1] Predicted: (37.38929, 126.33118) | Target: (33.55000, 126.55000) | ΔLat: 3.83929, ΔLon: 0.21882
[Step 11] Predicted: (37.10891, 126.16767) | Target: (33.55000, 126.55000) | ΔLat: 3.55891, ΔLon: 0.38233
[Step 21] Predicted: (37.03115, 126.12748) | Target: (33.55000, 126.55000) | ΔLat: 3.48115, ΔLon: 0.42252
[Step 31] Predicted: (36.98857, 126.10006) | Target: (33.55000, 126.55000) | ΔLat: 3.43857, ΔLon: 0.44994
[Step 41] Predicted: (36.94251, 126.07428) | Target: (33.55000, 126.55000) | ΔLat: 3.39251, ΔLon: 0.47572
[Step 51] Predicted: (36.88468, 126.04324) | Target: (33.55000, 126.55000) | ΔLat: 3.33468, ΔLon: 0.50676
[Step 61] Predicted: (36.81230, 126.00491) | Target: (33.55000, 126.55000) | ΔLat: 3.26230, ΔLon: 0.54509
[Step 71] Predicted: (36.72229, 125.95927) | Target: (33.55000, 126.55000) | ΔLat: 3.17229, ΔLon: 0.59073
[Step 81] Predicted: (36.61604, 125.90851) | Target: (33.55000, 126.55000) | ΔLat: 3.06604, ΔLon: 0.64149
[Step 91] Predicted: (36.51114, 125.86721) | Ta

In [134]:
def inverse_transform_preds(preds, scaler):
    """
    역정규화를 수행하여 원래의 값으로 변환
    preds: 예측된 값들 (numpy 배열), shape: (steps, output_size)
    scaler: 학습에 사용된 MinMaxScaler
    """
    # 위도, 경도, SOG, COG, heading 값만 역정규화
    preds_unscaled = preds.copy()  # 예측된 값을 복사

    # 위도, 경도, SOG, COG, heading을 역정규화
    preds_unscaled[:, :5] = scaler.inverse_transform(preds_unscaled[:, :5])  # 역정규화
    return preds_unscaled

In [135]:
import folium
from folium.plugins import AntPath

def visualize_route(initial_seq, preds_inverse, dest_lat, dest_lon):
    """
    예측된 경로를 시각화하는 함수
    initial_seq: 초기 입력 시퀀스 (numpy 배열), shape: (10, 6)
    preds_inverse: 역정규화된 예측 결과, shape: (steps, 4)
    dest_lat, dest_lon: 목적지 좌표
    """
    # 초기 위치
    start = initial_seq[0, 0][:5]
    start = scaler.inverse_transform([start])
    start_lat = start[0][0] 
    start_lon = start[0][1]
    # 지도 생성 (출발지와 목적지가 모두 보이도록 설정)
    route_map = folium.Map(location=[start_lat, start_lon], zoom_start=6)

    # 시작점, 목적지 마커 추가
    folium.Marker([start_lat, start_lon], tooltip='Start', icon=folium.Icon(color='green')).add_to(route_map)
    folium.Marker([dest_lat, dest_lon], tooltip='Destination', icon=folium.Icon(color='red')).add_to(route_map)

    # 예측 경로
    route_coords = [[lat, lon] for lat, lon in preds_inverse[:, :2]]  # 위도, 경도만 사용
    # 예측 경로를 PolyLine으로 시각화
    folium.PolyLine(route_coords, color='blue', weight=3, tooltip="Predicted Route").add_to(route_map)

    # 예측 경로에 애니메이션 효과 추가
    AntPath(route_coords).add_to(route_map)

    return route_map

In [136]:
# 예측 결과를 역정규화 후 시각화하는 전체 코드
def predict_and_visualize(model, initial_seq, dest_lat, dest_lon, scaler, max_steps=5000, distance_threshold=0.1):
    preds = predict_autoregressive(model, initial_seq, dest_lat, dest_lon, scaler, max_steps, distance_threshold)
    # 역정규화
    preds = preds.squeeze(1)
    preds_inverse = inverse_transform_preds(preds, scaler)
    # 예측 경로 시각화
    route_map = visualize_route(initial_seq.numpy(), preds_inverse, dest_lat, dest_lon)

    return route_map

In [137]:
# 예시: 초기 시퀀스와 목적지 좌표로 예측 및 시각화
route_map = predict_and_visualize(model, test_input_seq, dest_lat=33.55, dest_lon=126.55, scaler=scaler)
route_map.save('predicted_route_map_v3.html')

[Step 1] Predicted: (37.38929, 126.33118) | Target: (33.55000, 126.55000) | ΔLat: 3.83929, ΔLon: 0.21882
[Step 11] Predicted: (37.10891, 126.16767) | Target: (33.55000, 126.55000) | ΔLat: 3.55891, ΔLon: 0.38233
[Step 21] Predicted: (37.03115, 126.12748) | Target: (33.55000, 126.55000) | ΔLat: 3.48115, ΔLon: 0.42252
[Step 31] Predicted: (36.98857, 126.10006) | Target: (33.55000, 126.55000) | ΔLat: 3.43857, ΔLon: 0.44994
[Step 41] Predicted: (36.94251, 126.07428) | Target: (33.55000, 126.55000) | ΔLat: 3.39251, ΔLon: 0.47572
[Step 51] Predicted: (36.88468, 126.04324) | Target: (33.55000, 126.55000) | ΔLat: 3.33468, ΔLon: 0.50676
[Step 61] Predicted: (36.81230, 126.00491) | Target: (33.55000, 126.55000) | ΔLat: 3.26230, ΔLon: 0.54509
[Step 71] Predicted: (36.72229, 125.95927) | Target: (33.55000, 126.55000) | ΔLat: 3.17229, ΔLon: 0.59073
[Step 81] Predicted: (36.61604, 125.90851) | Target: (33.55000, 126.55000) | ΔLat: 3.06604, ΔLon: 0.64149
[Step 91] Predicted: (36.51114, 125.86721) | Ta